In [10]:
import numpy as np
import cProfile
from imp import reload

from info_measures import spatial as ss
#import scipy.spatial as ss
from info_measures.numpy import kraskov_stoegbauer_grassberger as ksg

# Trees

In [193]:
x = np.random.randn(10000, 1)

In [194]:
xtree = ss.cKDTree(x)

In [150]:
%timeit xtree.query(x, 3+1, p=np.inf, n_jobs=1)

7.8 ms ± 23.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [151]:
%timeit xtree.query(x, 3+1, p=np.inf, n_jobs=-1)

2.9 ms ± 44 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [152]:
dvec = xtree.query(x, 3+1, p=np.inf, n_jobs=-1)[0][:, 3]

In [158]:
%timeit [len(xtree.query_ball_point(point, 50*dist - 1e-15, p=float('inf'))) for point, dist in zip(x, dvec)]

155 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [159]:
%timeit [len(pts) for pts in xtree.query_ball_point(x, 50*dvec - 1e-15, p=float('inf'), n_jobs=-1)]

308 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [160]:
%timeit [len(pts) for pts in xtree.query_ball_point(x, 50*dvec - 1e-15, p=float('inf'), n_jobs=1)]

315 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [205]:
import numpy as np
import scipy.spatial as ss
x = np.random.randn(10000, 1)
xtree = ss.cKDTree(x)

In [206]:
%timeit [xtree.query_ball_point(point, .002, p=np.inf) for point in x]

55.9 ms ± 827 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [207]:
%timeit xtree.query_ball_point(x, .002, p=np.inf)

41.3 ms ± 750 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [208]:
r1 = [xtree.query_ball_point(point, .002, p=np.inf) for point in x]
r2 = xtree.query_ball_point(x, .002, p=np.inf)

In [211]:
all([np.allclose(sorted(rx1), sorted(rx2)) for rx1, rx2 in zip(r1, r2)])

True

In [192]:
%timeit xtree.query_ball_tree(xtree, 2, p=np.inf)

14.6 s ± 97.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [189]:
xtree.query_ball_point(x, .2, p=np.inf)

array([list([0, 3672, 4298, 5119, 5595, 6186, 7153, 7678, 9173]),
       list([1, 4071]), list([2, 2592, 5196, 6122, 8355, 8631, 8944]), ...,
       list([366, 690, 815, 934, 1284, 1602, 1756, 2034, 2343, 3899, 4032, 4377, 4584, 4620, 5156, 5298, 6109, 6684, 6699, 8954, 9174, 9997]),
       list([35, 260, 339, 349, 580, 1477, 4451, 4648, 5009, 5043, 5075, 5213, 5504, 6358, 6442, 6649, 6971, 8050, 8082, 8926, 9289, 9917, 9998]),
       list([570, 883, 1035, 1521, 1610, 1965, 2144, 2191, 3270, 3636, 3917, 4021, 4596, 4609, 4666, 5650, 6004, 6706, 7097, 8122, 9062, 9234, 9417, 9999])], dtype=object)

In [10]:
assert all([np.allclose(sorted(x), sorted(y)) for x, y in zip(xtree.query_ball_point(x, dvec - 1e-15, p=float('inf'), n_jobs=-1),
                                            [xtree.query_ball_point(point, dist - 1e-15, p=float('inf')) for point, dist in zip(x, dvec)])])

In [5]:
nns1 = xtree.query_ball_point(x, 1., p=np.inf, n_jobs=1)

In [8]:
nns2 = xtree.query_ball_tree(xtree, 1., p=np.inf)

In [13]:
all([np.allclose(sorted(x), sorted(y)) for x, y in zip(nns1, nns2)])

True

In [21]:
%timeit xtree.query_ball_point(x, 1., p=np.inf, n_jobs=-1)

1min 32s ± 1.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit xtree.query_ball_tree(xtree, 1., p=np.inf)

1min 58s ± 3.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


# digamma

In [19]:
%timeit ksg.avgdigamma1(xtree, x, dvec)
cProfile.run('ksg.avgdigamma1(xtree, x, dvec)')

113 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
         100005 function calls in 0.127 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.127    0.127 <string>:1(<module>)
        1    0.033    0.033    0.127    0.127 kraskov_stoegbauer_grassberger.py:175(avgdigamma1)
    20000    0.007    0.000    0.022    0.000 numeric.py:463(asarray)
    20000    0.006    0.000    0.024    0.000 numeric.py:586(ascontiguousarray)
        1    0.000    0.000    0.127    0.127 {built-in method builtins.exec}
    10001    0.001    0.000    0.001    0.000 {built-in method builtins.len}
    40000    0.033    0.000    0.033    0.000 {built-in method numpy.core.multiarray.array}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
    10000    0.048    0.000    0.094    0.000 {method 'query_ball_point' of 'scipy.spatial.ckdtree.cKDTree' objects}




In [20]:
%timeit ksg.avgdigamma1_v(xtree, x, dvec, n_jobs=-1)
cProfile.run('ksg.avgdigamma1_v(xtree, x, dvec, n_jobs=-1)')

34.2 ms ± 814 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
         50425 function calls in 0.041 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.041    0.041 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:34(_prod)
        9    0.000    0.000    0.000    0.000 _weakrefset.py:38(_remove)
        9    0.000    0.000    0.000    0.000 _weakrefset.py:81(add)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2408(prod)
        2    0.000    0.000    0.000    0.000 index_tricks.py:566(__init__)
        2    0.000    0.000    0.000    0.000 index_tricks.py:574(__iter__)
    20002    0.006    0.000    0.009    0.000 index_tricks.py:585(__next__)
        1    0.012    0.012    0.041    0.041 kraskov_stoegbauer_grassberger.py:198(avgdigamma1_v)
        4    0.000    0.000    0.000    0.000 numeric.py:463(asarray)
        2    0.000    0.000    0.0

# MI

In [119]:
x = np.random.randn(100000, 2)

In [125]:
reload(ksg)
ksg_e = ksg.MutualInformation(x[:,[0]], x[:,[1]])
k=3
dvec = ksg_e.tree.query(ksg_e.Z, k + 1, p=float('inf'))[0][:,k]

In [126]:
ksg_e.mutual_information()

-0.0034021263439374394

In [130]:
[len(xp) for xp in ksg_e.Xtree.query_ball_point(ksg_e.X, dvec)]

[323,
 549,
 486,
 1120,
 983,
 583,
 468,
 571,
 710,
 477,
 1314,
 287,
 247,
 1029,
 588,
 553,
 260,
 402,
 688,
 663,
 266,
 487,
 578,
 406,
 329,
 597,
 557,
 154,
 660,
 41,
 1258,
 546,
 985,
 479,
 745,
 1698,
 833,
 493,
 584,
 487,
 240,
 1099,
 178,
 170,
 212,
 372,
 366,
 386,
 478,
 451,
 473,
 498,
 216,
 425,
 307,
 296,
 673,
 249,
 1129,
 474,
 298,
 442,
 665,
 805,
 914,
 407,
 681,
 274,
 562,
 588,
 525,
 417,
 376,
 504,
 635,
 568,
 1315,
 313,
 213,
 337,
 238,
 456,
 635,
 279,
 366,
 585,
 307,
 2280,
 390,
 739,
 862,
 123,
 461,
 526,
 455,
 529,
 725,
 276,
 836,
 539,
 508,
 598,
 929,
 580,
 548,
 557,
 530,
 1811,
 299,
 522,
 485,
 388,
 620,
 1351,
 567,
 955,
 856,
 887,
 375,
 184,
 355,
 919,
 1442,
 629,
 486,
 1193,
 637,
 514,
 378,
 656,
 520,
 379,
 767,
 758,
 351,
 213,
 443,
 245,
 262,
 2074,
 666,
 618,
 756,
 574,
 527,
 304,
 450,
 577,
 327,
 616,
 285,
 620,
 1224,
 916,
 423,
 491,
 437,
 524,
 145,
 752,
 267,
 818,
 509,
 707,
 3

In [127]:
%timeit ksg.avgdigamma1(ksg_e.Xtree, ksg_e.X, dvec)

2.42 s ± 26.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
ksg_e.Xtree, ksg_e.X, dvec, n_jobs=1)

In [128]:
%timeit ksg.avgdigamma1_v(ksg_e.Xtree, ksg_e.X, dvec, n_jobs=1)

11.8 s ± 31.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [102]:
ksg_e.mutual_information(v=True)

-0.0012156443371136305

In [ ]:
%timeit ksg_e.mutual_information(k=3)

In [ ]:
%timeit ksg_e.mutual_information(k=3, v=True)

In [48]:
cProfile.run('ksg_e.mutual_information(n_jobs=1)')

         200030 function calls in 0.375 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.375    0.375 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:25(_amax)
        1    0.000    0.000    0.000    0.000 _methods.py:34(_prod)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:138(reshape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:1565(shape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2174(amax)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2408(prod)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:55(_wrapfunc)
        1    0.000    0.000    0.375    0.375 kraskov_stoegbauer_grassberger.py:111(mutual_information)
        2    0.090    0.045    0.362    0.181 kraskov_stoegbauer_grassberger.py:154(avgdigamma1)
        1    0.000    0.000    0.000    0.000 numeric.py:2135(isscalar)
    400

In [49]:
cProfile.run('ksg_e.mutual_information(n_jobs=1, v=True)')

         100114 function calls in 0.810 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.809    0.809 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:25(_amax)
        3    0.000    0.000    0.000    0.000 _methods.py:34(_prod)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:138(reshape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:1565(shape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2174(amax)
        3    0.000    0.000    0.000    0.000 fromnumeric.py:2408(prod)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:55(_wrapfunc)
        4    0.000    0.000    0.000    0.000 index_tricks.py:566(__init__)
        4    0.000    0.000    0.000    0.000 index_tricks.py:574(__iter__)
    40004    0.015    0.000    0.021    0.000 index_tricks.py:585(__next__)
        1    0.002    0.002    0.809    0.809 krasko